In [132]:
# from espnapi.espn_api.football import League
from espn_api.football import League
import pandas as pd
import plotly.express as px
# Init

#fan's - 42936131
#mine - 829963546
league_id = 829963546
year = 2022
league = League(league_id=league_id, year=year)



## Lets get the draft picks from each team

In [133]:
df = pd.DataFrame()

teams = []
players = []
id = []
round_num = []
round_pick = []


for pick in league.draft:
    teams.append(pick.team.team_name)
    players.append(pick.playerName)
    round_num.append(pick.round_num)
    round_pick.append(pick.round_pick)
    id.append(pick.playerId)
    
df['Team'] = teams
df['Player'] = players
df['ID'] = id
df['RoundPick'] = round_pick
df['RoundNum'] = round_num
df.to_csv('Data/team_players.csv')
df.head(10)

,Team,Player,ID,RoundPick,RoundNum
0,P-Town CASH KEVIN,Jonathan Taylor,4242335,1,1
1,Diggy Wit it,Christian McCaffrey,3117251,2,1
2,My Team A Bust,Austin Ekeler,3068267,3,1
3,Eugene Tweakers,Cooper Kupp,2977187,4,1
4,Couv Reigning Champ,Dalvin Cook,3116593,5,1
5,Gorilla Guys,Derrick Henry,3043078,6,1
6,I dont know what im doing,Leonard Fournette,3115364,7,1
7,Cougar Town Zach Wilson,Deebo Samuel,3126486,8,1
8,No Nyugen Will Win,Najee Harris,4241457,9,1
9,Just-in-N-out Drive thru,Justin Jefferson,4262921,10,1


# Get projected points

In [134]:
players = []
expected = []
actual = []
rank = []
nfl_team = []
for team in league.teams:
    for player in team.roster:
        players.append(player.name)
        expected.append(player.projected_total_points)
        actual.append(player.total_points)
        rank.append(player.posRank)
        nfl_team.append(player.proTeam)

points_df = pd.DataFrame()
points_df['Player'] = players
points_df['Expected_Points'] = expected
points_df['Actual_Points'] = actual
points_df['Rank'] = rank
points_df['NFLTeam'] = nfl_team



# Calculate Differences

In [135]:
points_df['Difference'] = points_df['Actual_Points'] - points_df['Expected_Points']
points_df = points_df.sort_values('Difference', ascending = False)

# Merge dataframes

In [136]:
combined_df = pd.merge(df, points_df, on = 'Player')
combined_df.head(10)

,Team,Player,ID,RoundPick,RoundNum,Expected_Points,Actual_Points,Rank,NFLTeam,Difference
0,P-Town CASH KEVIN,Jonathan Taylor,4242335,1,1,316.99,112.00,24,IND,-204.99
1,Diggy Wit it,Christian McCaffrey,3117251,2,1,309.94,201.76,3,SF,-108.18
2,My Team A Bust,Austin Ekeler,3068267,3,1,284.73,230.10,1,LAC,-54.63
3,Eugene Tweakers,Cooper Kupp,2977187,4,1,312.91,201.40,5,LAR,-111.51
4,Couv Reigning Champ,Dalvin Cook,3116593,5,1,258.49,156.50,11,MIN,-101.99
5,Gorilla Guys,Derrick Henry,3043078,6,1,264.47,203.06,2,TEN,-61.41
6,I dont know what im doing,Leonard Fournette,3115364,7,1,258.74,154.70,12,TB,-104.04
7,Cougar Town Zach Wilson,Deebo Samuel,3126486,8,1,261.46,130.00,21,SF,-131.46
8,No Nyugen Will Win,Najee Harris,4241457,9,1,265.48,124.96,20,PIT,-140.52
9,Just-in-N-out Drive thru,Justin Jefferson,4262921,10,1,287.80,215.52,4,MIN,-72.28


In [137]:
#normalize
combined_df['Normalized_Difference'] = combined_df['Difference'] /combined_df['Difference'].abs().max()
  

In [138]:
combined_df

,Team,Player,ID,RoundPick,RoundNum,Expected_Points,Actual_Points,Rank,NFLTeam,Difference,Normalized_Difference
0,P-Town CASH KEVIN,Jonathan Taylor,4242335,1,1,316.99,112.00,24,IND,-204.99,-0.912933
1,Diggy Wit it,Christian McCaffrey,3117251,2,1,309.94,201.76,3,SF,-108.18,-0.481785
2,My Team A Bust,Austin Ekeler,3068267,3,1,284.73,230.10,1,LAC,-54.63,-0.243297
3,Eugene Tweakers,Cooper Kupp,2977187,4,1,312.91,201.40,5,LAR,-111.51,-0.496615
4,Couv Reigning Champ,Dalvin Cook,3116593,5,1,258.49,156.50,11,MIN,-101.99,-0.454218
...,...,...,...,...,...,...,...,...,...,...,...
113,Eugene Tweakers,Cowboys D/ST,-16006,4,15,106.49,109.00,2,DAL,2.51,0.011178
114,Couv Reigning Champ,Harrison Butker,3055899,5,15,145.15,47.00,29,KC,-98.15,-0.437116
115,Cougar Town Zach Wilson,Brian Robinson Jr.,4241474,8,15,102.87,52.90,52,WSH,-49.97,-0.222544
116,No Nyugen Will Win,Christian Watson,4248528,2,16,108.76,80.50,53,GB,-28.26,-0.125857


# Now get their actual players

In [139]:
teams = []
players = []
projected = []
actual = []
stats = []
ranks = []
weeks = []
current_df = pd.DataFrame()
for i in range(1, 13):
    league.load_roster_week(i)
    for team in league.teams:
        for player in team.roster:
            weeks.append(i)
            teams.append(team.team_name)
            players.append(player.name)
            projected.append(player.projected_total_points)
            actual.append(player.total_points)
            ranks.append(player.posRank)

current_df['Team'] = teams
current_df['Player'] = players
current_df['Week'] = weeks
current_df['Rank'] = ranks
current_df['Expected_Points'] = projected
current_df['Actual_Points'] = actual
current_df['Week'] = weeks
current_df['Difference'] = current_df['Actual_Points'] - current_df['Expected_Points']

In [140]:
current_df.to_csv('Data/current_players.csv')

In [141]:
combined_df.groupby('Team').mean()

,ID,RoundPick,RoundNum,Expected_Points,Actual_Points,Rank,Difference,Normalized_Difference
Team,,,,,,,,
Cougar Town Zach Wilson,3.490025e+06,5.916667,7.083333,196.603333,99.983333,27.750000,-96.620000,-0.430302
Couv Reigning Champ,3.366947e+06,5.500000,7.700000,208.813000,118.552000,21.100000,-90.261000,-0.401982
Diggy Wit it,2.472610e+06,5.500000,7.333333,202.690833,116.321667,25.083333,-86.369167,-0.384649
Eugene Tweakers,2.832234e+06,5.384615,7.153846,222.665385,108.844615,32.538462,-113.820769,-0.506906
Gorilla Guys,2.992381e+06,5.545455,6.000000,213.820909,137.067273,26.818182,-76.753636,-0.341826
I dont know what im doing,1.888404e+06,5.090909,6.545455,195.667273,125.383636,19.909091,-70.283636,-0.313012
Just-in-N-out Drive thru,3.285116e+06,5.090909,6.272727,206.675455,150.721818,10.545455,-55.953636,-0.249192
My Team A Bust,3.666029e+06,5.500000,8.500000,193.085000,128.234286,21.214286,-64.850714,-0.288816
No Nyugen Will Win,3.494793e+06,5.230769,8.461538,193.516154,119.118462,21.461538,-74.397692,-0.331334


In [142]:
print('Done')

Done
